In [ ]:
# Load the SQL extension for Jupyter notebooks
%load_ext sql

# Import necessary libraries
import sqlite3
from sqlalchemy import create_engine
from IPython.display import Image, display
import pandas as pd
import pymysql
import os, sys 
from pathlib import Path
sys.path.append("../../scripts")
from db_utils import *    


In [ ]:
path = "amirmotefaker/supply-chain-dataset"
result = fetch_kaggle_dataset_by_path(path, temp_dir=".temp_kaggle")

📦 Fetching Kaggle dataset: amirmotefaker/supply-chain-dataset
Dataset URL: https://www.kaggle.com/datasets/amirmotefaker/supply-chain-dataset


Dropdown(description='Select File:', options=('supply_chain_data.csv',), value='supply_chain_data.csv')

Button(description='📥 Load Selected File', style=ButtonStyle())

Output()

🔄 Select a file to load individually. You can rerun to load others.


In [4]:
supply_df =result['raw_df']
supply_df

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,haircare,SKU95,77.903927,65,672,7386.363944,Unknown,15,14,26,...,Mumbai,18,450,26,58.890686,Pending,1.210882,Air,Route A,778.864241
96,cosmetics,SKU96,24.423131,29,324,7698.424766,Non-binary,67,2,32,...,Mumbai,28,648,28,17.803756,Pending,3.872048,Road,Route A,188.742141
97,haircare,SKU97,3.526111,56,62,4370.916580,Male,46,19,4,...,Mumbai,10,535,13,65.765156,Fail,3.376238,Road,Route A,540.132423
98,skincare,SKU98,19.754605,43,913,8525.952560,Female,53,1,27,...,Chennai,28,581,9,5.604691,Pending,2.908122,Rail,Route A,882.198864


In [19]:
supply_df.columns

Index(['Product type', 'SKU', 'Price', 'Availability',
       'Number of products sold', 'Revenue generated', 'Customer demographics',
       'Stock levels', 'Lead times', 'Order quantities', 'Shipping times',
       'Shipping carriers', 'Shipping costs', 'Supplier name', 'Location',
       'Lead time', 'Production volumes', 'Manufacturing lead time',
       'Manufacturing costs', 'Inspection results', 'Defect rates',
       'Transportation modes', 'Routes', 'Costs'],
      dtype='object')

**`fact_product_performance`**

Contains measurable, quantitative data.

| Column                  | Notes                              |
| ----------------------- | ---------------------------------- |
| `sku_id` (FK)           | Foreign key to `dim_product`       |
| `customer_demo_id` (FK) | Foreign key to `dim_customer_demo` |
| `supplier_id` (FK)      | Foreign key to `dim_supplier`      |
| `shipping_id` (FK)      | Foreign key to `dim_shipping`      |
| `manufacturing_id` (FK) | Foreign key to `dim_manufacturing` |
| `products_sold`         | Fact measure                       |
| `revenue_generated`     | Fact measure                       |
| `stock_levels`          | Fact measure                       |
| `lead_times`            | Fact measure (days)                |
| `order_quantities`      | Fact measure                       |
| `costs`                 | Fact measure (total cost)          |

---

`dim_product`

| Column         | Notes                    |
| -------------- | ------------------------ |
| `sku_id` (PK)  | Primary Key (SKU code)   |
| `product_type` | e.g., haircare, skincare |
| `price`        | Unit price               |
| `availability` | Stock availability flag  |

---

`dim_customer_demo`

| Column                  |
| ----------------------- |
| `customer_demo_id` (PK) |
| `customer_demographics` |

---

`dim_supplier`

| Column             |
| ------------------ |
| `supplier_id` (PK) |
| `supplier_name`    |
| `location`         |

---

`dim_shipping`

| Column                 |
| ---------------------- |
| `shipping_id` (PK)     |
| `shipping_carriers`    |
| `shipping_times`       |
| `shipping_costs`       |
| `transportation_modes` |
| `routes`               |

---

`dim_manufacturing`

| Column                    |
| ------------------------- |
| `manufacturing_id` (PK)   |
| `production_volumes`      |
| `manufacturing_lead_time` |
| `manufacturing_costs`     |
| `inspection_results`      |
| `defect_rates`            |

---

### **Products Table**
Stores core product details.  
**Columns:**  
- `product_id` (Primary Key)  
- `product_type`  
- `SKU`  
- `price`  
- `availability`  
- `stock_levels`  

### **Sales Table**  
Tracks product sales and revenue.  
**Columns:**  
- `sale_id` (Primary Key)  
- `product_id` (Foreign Key referencing Products)  
- `number_of_products_sold`  
- `revenue_generated`  
- `customer_id` (Foreign Key referencing Customers)  

### **Customers Table**  
Stores demographic information.  
**Columns:**  
- `customer_id` (Primary Key)  
- `customer_demographics`  

### **Orders Table**  
Tracks product orders.  
**Columns:**  
- `order_id` (Primary Key)  
- `product_id` (Foreign Key referencing Products)  
- `order_quantities`  
- `lead_times`  

### **Shipping Table**  
Manages logistics and carriers.  
**Columns:**  
- `shipping_id` (Primary Key)  
- `order_id` (Foreign Key referencing Orders)  
- `shipping_times`  
- `shipping_carriers`  
- `shipping_costs`  
- `transportation_modes`  
- `routes`  

### **Suppliers Table**  
Stores supplier details.  
**Columns:**  
- `supplier_id` (Primary Key)  
- `supplier_name`  
- `location`  
- `lead_time`  

### **Manufacturing Table**  
Tracks production metrics.  
**Columns:**  
- `manufacturing_id` (Primary Key)  
- `supplier_id` (Foreign Key referencing Suppliers)  
- `production_volumes`  
- `manufacturing_lead_time`  
- `manufacturing_costs`  

### **Quality Control Table**  
Monitors inspection results.  
**Columns:**  
- `inspection_id` (Primary Key)  
- `manufacturing_id` (Foreign Key referencing Manufacturing)  
- `inspection_results`  
- `defect_rates`  

### **Transportation Table**  
Handles logistics-related costs.  
**Columns:**  
- `transportation_id` (Primary Key)  
- `shipping_id` (Foreign Key referencing Shipping)  
- `transportation_modes`  
- `routes`  
- `costs`
Lookup table: (suuply_lookup)
- takes in all primary keys